In [1]:
# using version 0.2.5

In [2]:
## conditional things? c1?

# Search Spaces
As the name suggests, a [Search Space](https://hyperopt.github.io/hyperopt/getting-started/search_spaces/) is a multi-dimensional space through which we will search. The objective of the search is to find an n-dimensional point which yields the optimal result. The optimal result is objectively quantified by an objective function (loss function). In other words, the objective functrion will accept an n-dimensional point and return a numeric representation of the "goodness" of the point. 

The hyperopt framework provides the mechanism to define the search space and the algorithms to conduct the search. The user must define the objective function and the coresponding search space.

## The basic problem
I struggled with the notion of a search space at first. It is an abstract concept and the official documentation is not as robust as I would have liked being new to the project/terminology. Additionally there are many examples out there tailored to many use cases which add to the difficulty of making inferences about the library's functionality.

Again, the search space is generic and polymorphic. Depending on the problem we are trying to solve the physcial characteristics of the search space (schema/topography) will change. To simplify the presentation of the material we will focus on a single problem: Choosing the best machine learning algorithm given a set of input data. As we progress through this notebook we will examine the different representations of the search space that are possible.

Before putting our hands on the code, lets take a moment to think about the the problem philosophically. The first, and most obvious, question to answer (variable in our search space) is "what model will we use". Imagine we have a choice between "model a" and "model b". It is very likely that these models are significatnly different and require different hyperparameters. This would be our second question to answer: "what hyperparameters will we select for the selected model". We might be interested in exploring different search methods. We might be interested in comparing different performance metrics.

With that being said, we now have a number of variables to think about... but how do we describe this space so hyperopt can search over it?

### The basic syntax
As mentioned earlier, there are a number of ways to skin a cat in terms of defining a search space. As we will see everything is going to boil down to an expression of some sort which defines the domain of a variable. 

```
space = expression(label, parameters)
```

Hyperopt has a number of built in expression types. It also has the ability extend the builtin expressions with custom pyll functions which wee will see later.

A point of note here: The label must be globally unique or you may see a *DuplicateLabel* Exception get raised. The label helps the hyperopt framework define a graph like structure through which the search is conducted. If the labels were not uniquely identified, the framework woudl not be able to understand what parameters it is looking for. This will become more clear as we look at the complex nexted examples

### The objective function
As stated earlier, the objective function is responsible for evaluating the "goodness" or "performance" of a model or function when it is provided with a particular parameter set. The evaluation is provided as a globally comparable metric or measurement (some type of number).

```
metric = objective(parameter_set)
```

Defining this measure is outside the scope of this notebook. At a high level, the function might be defined as:

```
def objective(parameters_set):

    // Create a machine learning model
    // Train the model
    // Test the model
    // return test results metric

```


For educational purposed we will instead default to a trivial measure. Our objective function will return a number representing the sum of the sample parameters it was provided from the search space. For example, consider the following:

```
metric = objective(0)
# metric == 0

metric = objective(1,2,3)
# metric == 6

```

As stated earlier, defining the objective function depends on the search space. If the schema of the search space changes, so must the objective function. As we go through our examples, we will see the objective function's definition change to fit our search space.

# 1. Parameter Expressions
Parameter expressions are the backbone of defining search spaces. There are two types of parameter expressions which we can leverage:
- Stochastic Expressions - Define the domain of a stoachastic random variable
- Pyll Expressions - Define a deterministic (non-stochastic) variable

## 1.1. Stochastic Parameter Expressions
Stochastic expressions allow us to define random variables according to a commpon parameterized probability distribution.

Currently the following are supported:
- choice
- pchoice
- randint
- uniform
- quniform
- quniformint
- loguniform
- qloguniform
- normal
- qnormal
- lognormal
- qlognormal

All of these parameter expressions are found in the hyperopt.hp module. After looking through the code and making some inferences, I believe that hp stands for hyper paremter. Thus these expressions are hyperparameter expressions.

We will take a closer look at these individually in the next few sections.

### 1.1.1. The Choice Expression
With the choice expression, we can define a choice as a set of posisble outcomes with no specific distribution attached to it.

#### 1.1.1.1 A Univariate Choice
Below we can define a seach space as a binary choice. In this simple case we are choosing between two models (generically named "model a" and "model b").

In [3]:
import hyperopt

space = hyperopt.hp.choice('my_choice', [
    {'name': 'model a'},
    {'name': 'model b'}
])

We can use the hyperopt framework to sable from this search space and examine the arguments that would be provided to our objective function.

In [4]:
print(hyperopt.pyll.stochastic.sample(space))
print(hyperopt.pyll.stochastic.sample(space))
print(hyperopt.pyll.stochastic.sample(space))

{'name': 'model a'}
{'name': 'model b'}
{'name': 'model a'}


We can define a loss function and search through our parameter space for the optimal value using the hyperopt framework:

In [12]:
import hyperopt
import numpy

# Define the objective function
def objective(args):
    print(args)
    if args["name"] == "model a":
        return 0
    else:
        return 1

# Define an object to keep track of the "trials" in the search path
trials = hyperopt.Trials()
    
# Optimize the search space and retrieve the index which points to the best points in the search space
optimal_args_index = hyperopt.fmin(objective, space, algo=hyperopt.tpe.suggest, max_evals=10, trials=trials, rstate= numpy.random.RandomState(42))
    
# Retrieve the resulting hyperparameter set from the search space using the index
optimal_hyperparams = hyperopt.space_eval(space, optimal_args_index)

# Print the results
print("=========================")
print("Optimal args index:")
print(optimal_args_index)
print("Best hyperparameters:")
print(optimal_hyperparams)

{'name': 'model b', 'x': 2, 'y': 3}                                            
{'name': 'model a', 'x': 6}                                                    
{'name': 'model a', 'x': 1}                                                    
{'name': 'model a', 'x': 4}                                                    
{'name': 'model b', 'x': 3, 'y': 3}                                            
{'name': 'model b', 'x': 1, 'y': 5}                                            
{'name': 'model a', 'x': 8}                                                    
{'name': 'model a', 'x': 6}                                                    
{'name': 'model a', 'x': 8}                                                    
{'name': 'model a', 'x': 2}                                                    
100%|███████████████████████| 10/10 [00:00<00:00, 79.36trial/s, best loss: 0.0]
Optimal args index:
{'model_a_x': 3, 'my_choice': 0}
Best hyperparameters:
{'name': 'model a', 'x': 6}


We can see that "model a" was selected as it yields the minimal results from the objective function.

**Note:** We see that the search algorithm has a lot of repetitions... This is because it the only boundary on the search is the max_evals parameter. We will look at optimizing the search algorithm later on. For example when we utilize the loss_threshold to allow for early termination.

Having a look at the trials opject we inspect it's type and the useful properties.

In [15]:
trials

In [16]:
dir(trials)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_dynamic_trials',
 '_exp_key',
 '_ids',
 '_insert_trial_docs',
 '_trials',
 'aname',
 'argmin',
 'assert_valid_trial',
 'asynchronous',
 'attachments',
 'average_best_error',
 'best_trial',
 'count_by_state_synced',
 'count_by_state_unsynced',
 'delete_all',
 'fmin',
 'idxs',
 'idxs_vals',
 'insert_trial_doc',
 'insert_trial_docs',
 'losses',
 'miscs',
 'new_trial_docs',
 'new_trial_ids',
 'refresh',
 'results',
 'source_trial_docs',
 'specs',
 'statuses',
 'tids',
 'trial_attachments',
 'trials',
 'vals',
 'view']

In [18]:
trials.trials[:2]

[{'state': 2,
  'tid': 0,
  'spec': None,
  'result': {'loss': 1.0, 'status': 'ok'},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'model_a_x': [],
    'model_b_x': [0],
    'model_b_y': [0],
    'my_choice': [0]},
   'vals': {'model_a_x': [],
    'model_b_x': [2],
    'model_b_y': [1],
    'my_choice': [1]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2021, 9, 5, 18, 27, 23, 935000),
  'refresh_time': datetime.datetime(2021, 9, 5, 18, 27, 23, 941000)},
 {'state': 2,
  'tid': 1,
  'spec': None,
  'result': {'loss': 0.0, 'status': 'ok'},
  'misc': {'tid': 1,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'model_a_x': [1],
    'model_b_x': [],
    'model_b_y': [],
    'my_choice': [1]},
   'vals': {'model_a_x': [3],
    'model_b_x': [],
    'model_b_y': [],
    'my_choice': [0]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datet

#### 1.1.1.2. A Nested Univariate Choice
In the example below, we define the search space as a choice between two models. Each model accepts a single hyperparameter which ranges in values.

In [6]:
import hyperopt
import numpy

# Define the search space
space = hyperopt.hp.choice('my_choice', [
    {
        'name': 'model a',
        'x': hyperopt.hp.choice('model_a_x', [0,2,4,6,8])
    },
    {
        'name': 'model b',
        'x': hyperopt.hp.choice('model_b_x', [1,3,5,7,9])
    }
])

# Define the objective function
def objective(args):
    x = args['x']
    return x

# Define an object to keep track of the "trials" in the search path
trials = hyperopt.Trials()
    
# Optimize the search space and retrieve the index which points to the best points in the search space
optimal_args_index = hyperopt.fmin(objective, space, algo=hyperopt.tpe.suggest, max_evals=10, trials=trials, rstate= numpy.random.RandomState(42))
    
# Retrieve the resulting hyperparameter set from the search space using the index
optimal_hyperparams = hyperopt.space_eval(space, optimal_args_index)

# Print the results
print("=========================")
print("Optimal args index:")
print(optimal_args_index)
print("Best hyperparameters:")
print(optimal_hyperparams)

100%|██████████████████████| 10/10 [00:00<00:00, 222.21trial/s, best loss: 0.0]
Optimal args index:
{'model_a_x': 0, 'my_choice': 0}
Best hyperparameters:
{'name': 'model a', 'x': 0}


#### 1.1.1.3. A multivariate nested choice
In the next example we will make things a bit more interesting. We will use models which accept a different set of hyper parameters (one of them being multivariate).

In [10]:
# Define the search space
space = hyperopt.hp.choice('my_choice', [
    {
        'name': 'model a',
        'x': hyperopt.hp.choice('model_a_x', [1,2,4,6,8])
    },
    {
        'name': 'model b',
        'x': hyperopt.hp.choice('model_b_x', [0,1,2,3,4]),
        'y': hyperopt.hp.choice('model_b_y', [0,3,5,7,9])        
    }
])

# Define the objective function
def objective(args):
    x = args['x']
    y = args['y'] if 'y' in args.keys() else 0
    return x + y

# Define an object to keep track of the "trials" in the search path
trials = hyperopt.Trials()
    
# Optimize the search space and retrieve the index which points to the best points in the search space
optimal_args_index = hyperopt.fmin(objective, space, algo=hyperopt.tpe.suggest, max_evals=1000, trials=trials, rstate= numpy.random.RandomState(42), loss_threshold=0.1)
    
# Retrieve the resulting hyperparameter set from the search space using the index
optimal_hyperparams = hyperopt.space_eval(space, optimal_args_index)

# Print the results
print("=========================")
print("Optimal args index:")
print(optimal_args_index)
print("Best hyperparameters:")
print(optimal_hyperparams)

 62%|███████████▋       | 618/1000 [00:05<00:03, 104.51trial/s, best loss: 0.0]
Optimal args index:
{'model_b_x': 0, 'model_b_y': 0, 'my_choice': 1}
Best hyperparameters:
{'name': 'model b', 'x': 0, 'y': 0}


**Note:** We had to significantly increase the mas_evals and add the loss threshold param to allow us to terminate early if the loss is less than the threshold supplied.